In [1158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import *
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import svm
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

*データセットの読み込み*

In [1159]:
# データセットの読み込み
csv_path = "../../../input/house-prices-advanced-regression-techniques/train.csv" # ファイル名（パス）を指定する
data = pd.read_csv(csv_path)
data = data.loc[:,['GrLivArea','YearBuilt','SalePrice']]

# 欠損値を埋める
data = data.fillna(df.median())

# 抽出
X = data.loc[:,['GrLivArea','YearBuilt']].values
y = data.loc[:,['SalePrice']].values
# 対数変換
X = np.log(X)
y = np.log(y)
# 分割
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 99)
y_train = y_train.flatten()
y_test = y_test.flatten()

In [1160]:
# 結果比較用のクラス
class Show_result():
    def __init__(self):
        self.cols = ["   RMSE train data","RMSE test data", "モデル名"]
        self.result_df = pd.DataFrame(index=[], columns=self.cols)

    def show_rsme(self, model_name, y_train, y_train_pred, y_test, y_pred):
        
        rsme1 = np.sqrt(mean_squared_error(y_train, y_train_pred))
        rsme2 = np.sqrt(mean_squared_error(y_test, y_pred))
        record = pd.Series([rsme1, rsme2, model_name], index=self.result_df.columns)
        
        self.result_df = self.result_df.append(record, ignore_index=True)
        
        print(self.result_df.sort_values('RMSE test data'))

***【問題1】ブレンディングのスクラッチ実装***

In [1161]:
class ScratchBlending():
    """
    ブレンディングのスクラッチ実装
    """
    def predict(self, X_test, model_list, weighted_average = None):
        
        # 予測値の変数（空）を宣言
        y_pred_list = np.empty((X_test.shape[0], len(model_list)))
        
        # モデルの数だけループ
        for i in range(len(model_list)):
        
            # それぞれのモデルより予測値を取得し、リストへ格納していく
            y_pred_list[:,i] = model_list[i].predict(X_test)
        
        # 重み付けをしない場合
        if weighted_average is None:
            # 予測値の平均を算出
            y_pred_list = np.average(y_pred_list, axis = 1)
        # 重み付けをする場合
        else:
            # 未実装
            pass
        
        return y_pred_list

In [1162]:
# 決定木
d_tree = tree.DecisionTreeRegressor(max_depth=5)
d_tree = d_tree.fit(X_train, y_train)
y_train_pred = d_tree.predict(X_train)
y_pred = d_tree.predict(X_test)

show_result = Show_result()
show_result.show_rsme("決定木", y_train, y_train_pred, y_test, y_pred)

      RMSE train data  RMSE test data モデル名
0            0.196419         0.18574  決定木


In [1163]:
# 線形回帰
lr = LinearRegression()
lr = lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
y_pred = lr.predict(X_test)

show_result.show_rsme("線形回帰", y_train, y_train_pred, y_test, y_pred)

      RMSE train data  RMSE test data  モデル名
0            0.196419        0.185740   決定木
1            0.219077        0.188343  線形回帰


In [1164]:
# SVM
svr = svm.SVR()
svr = svr.fit(X_train, y_train)
y_train_pred = svr.predict(X_train)
y_pred = svr.predict(X_test)

show_result.show_rsme("SVM", y_train, y_train_pred, y_test, y_pred)

      RMSE train data  RMSE test data  モデル名
0            0.196419        0.185740   決定木
1            0.219077        0.188343  線形回帰
2            0.215512        0.189379   SVM


In [1165]:
# ランダムフォレスト
rfr = RandomForestRegressor()
rfr = rfr.fit(X_train, y_train)
y_train_pred = rfr.predict(X_train)
y_pred = rfr.predict(X_test)

show_result.show_rsme("ランダムフォレスト", y_train, y_train_pred, y_test, y_pred)

      RMSE train data  RMSE test data       モデル名
3            0.084371        0.180743  ランダムフォレスト
0            0.196419        0.185740        決定木
1            0.219077        0.188343       線形回帰
2            0.215512        0.189379        SVM


In [1166]:
# ブレンディング
model_list = [d_tree, lr, svr, rfr] # 決定木、線形回帰、SVM、ランダムフォレスト
scr_bl = ScratchBlending()
y_train_pred = scr_bl.predict(X_train, model_list)
y_pred = scr_bl.predict(X_test, model_list)

show_result.show_rsme("ブレンディング（決定木、線形回帰、SVM、ランダムフォレスト）", y_train, y_train_pred, y_test, y_pred)

      RMSE train data  RMSE test data                             モデル名
4            0.168740        0.175593  ブレンディング（決定木、線形回帰、SVM、ランダムフォレスト）
3            0.084371        0.180743                        ランダムフォレスト
0            0.196419        0.185740                              決定木
1            0.219077        0.188343                             線形回帰
2            0.215512        0.189379                              SVM


***【問題2】バギングのスクラッチ実装***

In [1167]:
class ScratchBugging():
    """
    バギングのスクラッチ実装
    
    model : モデル
    n_estimators : 学習回数
    train_samples : 学習時にトレインデータから抽出するデータの割合
    """
    def __init__(self, model, n_estimators = 50, train_samples = 0.8):
        # ハイパーパラメータを属性として記録
        self.model = model
        self.n_estimators = n_estimators
        self.train_samples = train_samples
        
        self.model_list = []
        
    def fit(self, X, y):

        # 学習回数の数だけループ
        for i in range(self.n_estimators):
            
            # データを分割
            X_train, _, y_train, _ = train_test_split(X, y, train_size = self.train_samples)
            # 学習
            model = self.model.fit(X_train, y_train)
            # 学習済モデルをリストへ追加
            self.model_list.append(model)

    def predict(self, X):
        
        # 予測値の変数（空）を宣言
        y_pred_list = np.empty((X.shape[0], self.n_estimators))
        
        # モデルの数だけループ
        for i, model in enumerate(self.model_list):
            
            # それぞれのモデルより予測値を取得し、リストへ格納していく
            y_pred_list[:,i] = model.predict(X)
        
        # 予測値の平均を算出
        y_pred_list = np.average(y_pred_list, axis = 1)
        
        return y_pred_list

In [1168]:
# バギング（決定木）
d_tree = tree.DecisionTreeRegressor(max_depth=5)
scr_bug = ScratchBugging(d_tree)
scr_bug.fit(X_train, y_train)
y_train_pred = d_tree.predict(X_train)
y_pred = d_tree.predict(X_test)

show_result.show_rsme("バギング（決定木）", y_train, y_train_pred, y_test, y_pred)

      RMSE train data  RMSE test data                             モデル名
4            0.168740        0.175593  ブレンディング（決定木、線形回帰、SVM、ランダムフォレスト）
3            0.084371        0.180743                        ランダムフォレスト
0            0.196419        0.185740                              決定木
5            0.200446        0.185921                        バギング（決定木）
1            0.219077        0.188343                             線形回帰
2            0.215512        0.189379                              SVM


***【問題3】スタッキングのスクラッチ実装***

In [1169]:
class ScratchStacking():
    """
    スタッキングのスクラッチ実装
    
    base_models : ベースモデルのリスト
    meta_model : メタモデル
    n_splits : クロスバリデーション時の分割数
    """
    def __init__(self, base_models, meta_model, n_splits=4):
        # ハイパーパラメータを属性として記録
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_splits = n_splits
    
    """
    fit
    学習を行う
    """
    def fit(self, X, y):
        
        # KFoldのインスタンス
        kf = KFold(n_splits = self.n_splits, shuffle = True)
        # oofの変数（空）
        oof = np.zeros([X.shape[0], len(self.base_models)])
        
        # ベースモデルの数だけループ
        for i, base_model in enumerate(self.base_models):
            
            # 学習済モデルの変数（空）
            learned_models_list = []
            
            # クロスバリデーションを行うループ
            for train_index, test_index in kf.split(X): 

                # ベースモデルで学習
                base_model.fit(X[train_index], y[train_index])
                # 学習済モデルをリストに追加
                learned_models_list.append(base_model)
                # 推定
                y_pred = base_model.predict(X[test_index])
                # 推定結果をoofに格納
                oof[test_index, i] = y_pred.flatten()
                
            # 学習済ベースモデルのリストでベースモデルを上書き
            self.base_models[i] = learned_models_list
        
        # oofを特徴量としてメタモデルで学習
        self.meta_model.fit(oof, y)
        
    """
    predict
    推定を行う
    """
    def predict(self, X):
        
        new_features = np.zeros([X.shape[0], len(self.base_models)])
        
        # ベースモデルの数だけループ
        for i, base_models in enumerate(self.base_models):
            
            # 予測値の変数（空）を宣言
            y_pred_list = np.empty((X.shape[0], len(base_models)))
            
            # ベースモデルの数だけループ
            for j, base_model in enumerate(base_models):
            
                # それぞれのモデルより予測値を取得し、リストへ格納していく
                y_pred_list[:,j] = base_model.predict(X)
            
            # 予測値の平均を算出し、新しい特徴量とする
            new_features[:,i] = np.average(y_pred_list, axis = 1)
            
        # 作成した特徴量でメタモデルにより推定
        y_pred = self.meta_model.predict(new_features)
        
        return y_pred

In [1170]:
# スタッキング
rfr = RandomForestRegressor()
model_list = [d_tree, lr, svr] # 決定木、線形回帰、SVR
scr_st = ScratchStacking(model_list, rfr) # ランダムフォレスト
scr_st.fit(X_train, y_train)
y_train_pred = scr_st.predict(X_train)
y_pred = scr_st.predict(X_test)

show_result.show_rsme("スタッキング（決定木、線形回帰、SVR。ランダムフォレスト）", y_train, y_train_pred, y_test, y_pred)

      RMSE train data  RMSE test data                             モデル名
4            0.168740        0.175593  ブレンディング（決定木、線形回帰、SVM、ランダムフォレスト）
3            0.084371        0.180743                        ランダムフォレスト
0            0.196419        0.185740                              決定木
5            0.200446        0.185921                        バギング（決定木）
1            0.219077        0.188343                             線形回帰
2            0.215512        0.189379                              SVM
6            0.185492        0.194540   スタッキング（決定木、線形回帰、SVR。ランダムフォレスト）
